In [1]:
import os.path
import pprint
import re
from functools import partial

import bw2calc as bc
import bw2data as bd
import bw2io as bi

# create Change in Biosphere Integrity method
> Functional diversity, BII [biodiversity intactness index]

> Mean Species Abundance (MSA) is used as a proxy of Biological Intactness Index (BII)
“The main difference between MSA and BII is that every hectare is given equal weight in MSA, 
whereas BII gives more weight to species rich areas.” (Alkemade et al. 2009)

> Hanafiah et al., 2012 provides (1-MSA) values for diffent land types present in SimaPro.
(1-MSA) values are used as the characterization factors to assess ""Biological footprint, LU"". 
(1-MSA) values are applied to land use inventory flows (m2a).

% of BII loss ~ % of (1-MSA) = 100 x (BF_LU+BF_CC)/At  
where,   
(1-MSA)               = Loss of mean species aboundance   
> (from 0: no loss of species; to 1: complete loss of species)  

MSA                   = Mean species abundance  
> (from 0: no species; to 1: undisturbed biodevirsity)  

BF_LU  [BII loss*m2a] = Biological footprint for Land Use  
BF_CC [BII loss*m2a]  = Biological footprint for Climate Change  
At        [m2yr]      = Earth’s total land (used in 1 yr) = 1.30E+14  

where  
BF_LU is calculated as SUM[Ai*(1-MSAi)] for ""i-th"" land use types  
> (CFs provided in Table 1 in Hanafiah et al. 2012)  

BF_CC = 0.27 * GWP   
> GWP    [kg CO2 eq] = Global Warming Potential  for 100-year time horizon
                        (calculated using ""ReCiPe 2016 (H) midpoint"" characterization factors)

The factor applied to GWP is calculated as 100 x (0.27/At) = `2.07692E-13`  
the factor applied to BF_LU is calculated as 100 x (1/At)  = `7.69231E-13`

In [2]:
bd.projects

Brightway2 projects manager with 13 objects:
	StepByStep
	bw2_class_2020_example_database
	bw2_class_2020_intro
	car comparison paper
	check_electricity_EES_paper
	check_impact_electricity
	default
	ecoinvent 3.6
	global_power_mix
	parameters - manual creation
	premise_scenarios
	simapro-ecoinvent-import
	trial-import_from_SimaPro
Use `projects.report()` to get a report on all projects.

In [3]:
bd.projects.set_current("ecoinvent 3.6")

In [4]:
bd.databases

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.5
	ecoinvent 3.6

## Biological footprint: Climate Change component
> import ReCiPe2016 Midpoint(H) method and correct the units

In [5]:
import bw_recipe_2016
from bw_recipe_2016 import GlobalWarming, extract_recipe, get_biosphere_database

In [6]:
data = extract_recipe()[2]

In [7]:
biosphere = get_biosphere_database()

In [87]:
gw = GlobalWarming(data, biosphere)

In [88]:
# gw.apply_strategies()
# will get a TypeError message, need to correct the "split_synonyms" strategy

In [89]:
multiple = re.compile(r"^(.*)\((.*)\)$")


def split_synonyms_corrected(data):
    """Split synonyms given in one string.

    Also makes sure ``synonyms`` exists, has no trailing whitespace,
    and is a list.
    E.g. ``dinitrogen oxide (nitrous oxide)`` to
    ``["dinitrogen oxide", "nitrous oxide"]``,"""
    for ds in data:
        for cf in ds["exchanges"]:
            if cf.get("synonyms"):
                match = multiple.match(str(cf["synonyms"]))  # CORRECTION: make it str
                if match:
                    cf["synonyms"] = [x.strip() for x in match.groups()]
                elif not isinstance(cf["synonyms"], list):
                    cf["synonyms"] = [cf["synonyms"].strip()]
            else:
                cf["synonyms"] = []
    return data

In [90]:
gw.apply_strategies(gw.strategies[:2])
gw.apply_strategy(split_synonyms_corrected)  # corrected strategy
gw.apply_strategies(gw.strategies[3:])
gw.statistics()

Applying strategy: fix_perspective_string
Applying strategy: generic_reformat
Applied 2 strategies in 0.00 seconds
Applying strategy: split_synonyms_corrected
Applying strategy: more_synonyms
Applying strategy: fix_unit_string
Applying strategy: name_matcher
Applying strategy: add_biomass_stock_cfs
Applying strategy: add_air_category
Applying strategy: complete_method_name
Applying strategy: match_multiple
Applying strategy: final_method_name
Applying strategy: check_duplicate_cfs
Applied 9 strategies in 0.03 seconds
3 methods
1083 cfs
519 unlinked cfs


(3, 1083, 519)

In [91]:
# which is previous version?
c = gw.compare_to_previous()

In [92]:
c.keys()

dict_keys(['found', 'missing', 'reference'])

In [93]:
[(key, len(c[key])) for key in c]

[('found', 39), ('missing', 173), ('reference', 40)]

In [94]:
# Present in previous ReCiPe, missing here
c["reference"].difference(c["found"])

{'dimethyl ether', 'ethane, 1,1,1-trichloro-, hcfc-140'}

In [95]:
# c["found"]
# c["missing"]
# c["reference"]

In [96]:
gw.drop_unlinked()
gw.statistics()

Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
3 methods
564 cfs
0 unlinked cfs


(3, 564, 0)

In [97]:
def change_unit_of_biofootprint_climate_change(data):
    """Change the unit of biological footprint climate change (BF_CC).

    Units change from kg CO2eq to % of BII loss.
    """
    total_land_earth = 1.30e14  # total Earth's area
    # (from IMAGE model https://models.pbl.nl/image/index.php/Download_packages)
    f_individualist = 0.05  # 20-year time horizon (Hanafiah et al. 2012, Table 1)
    f_hierarchist = 0.27  # 100-year time horizon (Hanafiah et al. 2012, Table 1)
    f_egalitarian = 3.87  # infinite time horizon (Hanafiah et al. 2012, Table 1)

    # check if CFs were already changed.
    for method in data:
        for exc in method["exchanges"]:
            if "carbon dioxide, fossil" in exc["name"] and exc["amount"] != 1:
                return data  # if cfs were already changed

    for method in data:
        if "individualist" in str(method["name"]).lower():
            for exc in method["exchanges"]:
                exc["amount"] = 100 * (
                    f_individualist * exc["amount"] / total_land_earth
                )
        elif "hierarchist" in str(method["name"]).lower():
            for exc in method["exchanges"]:
                exc["amount"] = 100 * (f_hierarchist * exc["amount"] / total_land_earth)
        elif "egalitarian" in str(method["name"]).lower():
            for exc in method["exchanges"]:
                exc["amount"] = 100 * (f_egalitarian * exc["amount"] / total_land_earth)
    return data

In [98]:
gw.apply_strategy(change_unit_of_biofootprint_climate_change)
gw.statistics()

Applying strategy: change_unit_of_biofootprint_climate_change
3 methods
564 cfs
0 unlinked cfs


(3, 564, 0)

In [99]:
def drop_duplicates(data):
    for method in data:
        for exc in method["exchanges"]:
            unique_exchanges = {
                (o["name"], o["categories"]): o for o in method["exchanges"]
            }
            method["exchanges"] = list(unique_exchanges.values())
    return data

In [103]:
def add_subcategory_methane_air(data):
    """Add subcategory  'urban air close to ground' to methane"""
    for method in data:
        for exc in method["exchanges"]:
            if "methane" == exc["name"] and "air" in str(exc["categories"]):
                exc["categories"] = ("air", "urban air close to ground")
    return data

In [104]:
gw.apply_strategy(add_subcategory_methane_air)

Applying strategy: add_subcategory_methane_air


In [105]:
gw.apply_strategy(drop_duplicates)
gw.statistics()

Applying strategy: drop_duplicates
3 methods
117 cfs
0 unlinked cfs


(3, 117, 0)

In [106]:
gw.write_excel("BII_climate_change component")

Wrote matching file to:
C:\Users\ViteksPC\AppData\Local\pylca\Brightway3\ecoinvent-36.d2073cbbe9aff3f7cc76c4f9a36d6789\output\lcia-matching-BII_climate_change-component.xlsx


## Biological footprint: Land Use component
> import BF_LU method and correct the units

In [25]:
method_category = "BiologicalFootprint_LU"

In [26]:
si = bi.SimaProLCIACSVImporter(
    filepath=os.path.join(
        r"C:\Users\ViteksPC\Documents\00-ETH_projects\AESAmethods\from_simapro"
        f"\PBsLCIAv072_{method_category}.csv"
    )
)

Extracted 1 methods in 0.01 seconds


In [27]:
# si.data[0].keys()

In [28]:
for method in si.data:
    print(method["name"], method["unit"])

('PBs-LCIA (baseline)', '(not PBs) Biological footprint, LU**') BIILoss.m2a


In [29]:
si.data[0]["exchanges"][0]  # .keys()

{'amount': 0.9,
 'CAS number': '',
 'categories': ('Raw', '(unspecified)'),
 'name': 'Occupation, agriculture',
 'unit': 'm2a'}

In [30]:
si.apply_strategies()
si.statistics()

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: link_iterable_by_fields
Applying strategy: match_subcategories
Applied 10 strategies in 0.43 seconds
1 methods
67 cfs
27 unlinked cfs


(1, 67, 27)

In [31]:
def drop_duplicates(data):
    for method in data:

        unique_exchanges = {
            (o["name"], o["categories"]): o for o in method["exchanges"]
        }
        method["exchanges"] = list(unique_exchanges.values())
        return data

In [32]:
link_by_name_and_categories = partial(
    bi.strategies.link_iterable_by_fields,
    other=bd.Database(bd.config.biosphere),
    kind="biosphere",
    fields=("name", "categories"),
)

In [33]:
def change_to_green_area(data):
    for method in data:
        for exc in method["exchanges"]:
            if "green areas" in exc["name"]:
                exc["name"] = re.sub("green areas", "green area", exc["name"])
                exc["categories"] = (exc["categories"][0], "land")

        return data


def update_occupation_forest_secondary(data):
    for method in data:
        for exc in method["exchanges"]:
            if "Occupation, forest, secondary" in exc["name"]:
                exc["name"] = re.sub("secondary", "secondary (non-use)", exc["name"])
                exc["categories"] = (exc["categories"][0], "land")

        return data


def update_occupation_urban(data):
    for method in data:
        for exc in method["exchanges"]:
            if "Occupation, urban" in exc["name"]:
                exc["name"] = re.sub(
                    "urban$", "urban/industrial fallow (non-use)", exc["name"]
                )
                exc["categories"] = (exc["categories"][0], "land")

        return data

In [34]:
si.apply_strategies(
    [
        change_to_green_area,
        update_occupation_forest_secondary,
        update_occupation_urban,
        drop_duplicates,
        link_by_name_and_categories,
    ]
)
si.statistics()

Applying strategy: change_to_green_area
Applying strategy: update_occupation_forest_secondary
Applying strategy: update_occupation_urban
Applying strategy: drop_duplicates
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 0.15 seconds
1 methods
61 cfs
24 unlinked cfs


(1, 61, 24)

In [35]:
# list(si.unlinked)

In [36]:
# from the 5 unlinked:
# 1 has amount 0, can be dropped - Carbon dioxide, in air
# 2 others "Carbon dioxide" and "Carbon monoxide" are not in biosphere3, BUT
# "Carbon dioxide, fossil"
# and "Carbon monoxide, fossil" and "Carbon monoxide, non-fossil" (with same CFs)
# have been linked (see check_equivalent_linked)
[(exc["name"], exc["categories"]) for exc in list(si.unlinked)]

[('Occupation, agriculture', ('natural resource',)),
 ('Occupation, annual crop, non-irrigated, diverse-intensive',
  ('natural resource',)),
 ('Occupation, annual crop, non-irrigated, fallow', ('natural resource',)),
 ('Occupation, annual crop, non-irrigated, monotone-intensive',
  ('natural resource',)),
 ('Occupation, annual crop, organic', ('natural resource',)),
 ('Occupation, dump site, benthos', ('natural resource',)),
 ('Occupation, forest', ('natural resource',)),
 ('Occupation, forest, natural', ('natural resource',)),
 ('Occupation, forest, primary', ('natural resource',)),
 ('Occupation, forest, used', ('natural resource',)),
 ('Occupation, lakes, artificial', ('natural resource',)),
 ('Occupation, pasture, man made, organic', ('natural resource',)),
 ('Occupation, permanent crop, fruit', ('natural resource',)),
 ('Occupation, permanent crop, fruit, extensive', ('natural resource',)),
 ('Occupation, permanent crop, fruit, intensive', ('natural resource',)),
 ('Occupation, p

In [37]:
def see_unlinked(imported_methods, hide_print=True):
    unlinked_exc_names = [
        exc_unlinked["name"] for exc_unlinked in list(imported_methods.unlinked)
    ]

    for ix in range(len(imported_methods.data)):
        lst = []
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            if exc["name"] in unlinked_exc_names:
                lst.append((exc["name"], exc["categories"], exc["amount"]))
        if not hide_print:
            for i in list(set(lst)):
                print("\t", i)
        print("\n")

    return list(set(lst))
    # if exc["amount"] != 0

In [38]:
def check_equivalent_linked(imported_methods, list_names):
    for ix in range(len(imported_methods.data)):
        print(imported_methods.data[ix]["name"][1])
        for exc in imported_methods.data[ix]["exchanges"]:
            for x in list_names:
                if x in exc["name"]:
                    print(
                        "\t", exc["name"], exc["categories"], exc["amount"],
                    )
        print("\n")

In [39]:
sorted(see_unlinked(si))

(not PBs) Biological footprint, LU**




[('Occupation, agriculture', ('natural resource',), 0.9),
 ('Occupation, annual crop, non-irrigated, diverse-intensive',
  ('natural resource',),
  0.9),
 ('Occupation, annual crop, non-irrigated, fallow',
  ('natural resource',),
  0.7),
 ('Occupation, annual crop, non-irrigated, monotone-intensive',
  ('natural resource',),
  0.9),
 ('Occupation, annual crop, organic', ('natural resource',), 0.7),
 ('Occupation, dump site, benthos', ('natural resource',), 0.0),
 ('Occupation, forest', ('natural resource',), 0.5),
 ('Occupation, forest, natural', ('natural resource',), 0.0),
 ('Occupation, forest, primary', ('natural resource',), 0.0),
 ('Occupation, forest, used', ('natural resource',), 0.5),
 ('Occupation, lakes, artificial', ('natural resource',), 1.0),
 ('Occupation, pasture, man made, organic', ('natural resource',), 0.3),
 ('Occupation, permanent crop, fruit', ('natural resource',), 0.9),
 ('Occupation, permanent crop, fruit, extensive', ('natural resource',), 0.7),
 ('Occupatio

In [40]:
b3 = bd.Database("biosphere3")
set(
    [
        (flow["name"], flow["categories"])
        for flow in b3
        if "Occupation," in flow["name"] and "artificial" in flow["name"]  # .lower()
        if "natural resource" in str(flow["categories"])
    ]
)

{('Occupation, lake, artificial', ('natural resource', 'land')),
 ('Occupation, river, artificial', ('natural resource', 'land'))}

In [41]:
len(
    [
        "Occupation, agriculture",
        "Occupation, annual crop, non-irrigated, diverse-intensive",
        "Occupation, annual crop, non-irrigated, fallow",
        "Occupation, annual crop, non-irrigated, monotone-intensive",
        "Occupation, annual crop, organic",
        "Occupation, dump site, benthos",
        "Occupation, forest",
        "Occupation, forest, natural",
        "Occupation, forest, primary",
        "Occupation, forest, used",
        "Occupation, lakes, artificial",
        "Occupation, pasture, man made, organic",
        "Occupation, permanent crop, fruit",
        "Occupation, permanent crop, fruit, extensive",
        "Occupation, permanent crop, fruit, intensive",
        "Occupation, permanent crop, vine",
        "Occupation, permanent crop, vine, extensive",
        "Occupation, permanent crop, vine, intensive",
        "Occupation, sea and ocean",
        "Occupation, traffic area",
        "Occupation, traffic area, road embankment",
        "Occupation, unknown",
        "Occupation, unspecified, used",
        "Occupation, water courses, artificial",
    ]
)

24

> check Occupation, forest, intensive ('natural resource', 'land') 0.8 or 0.5

In [42]:
check_equivalent_linked(si, ["artificial"])

(not PBs) Biological footprint, LU**
	 Occupation, lakes, artificial ('natural resource',) 1.0
	 Occupation, water courses, artificial ('natural resource',) 1.0
	 Occupation, river, artificial ('natural resource', 'land') 1.0




In [43]:
si.drop_unlinked()
si.statistics()

Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
1 methods
37 cfs
0 unlinked cfs


(1, 37, 0)

In [44]:
def change_unit_of_biofootprint_land_use(data):
    """Change the unit of biological footprint land use (BF_LU).
    
    Units change from loss MSA to % of BII loss.
    """
    total_land_earth = 1.30e14  # total Earth's area
    # (from IMAGE model https://models.pbl.nl/image/index.php/Download_packages)

    # check if CFs were already changed.
    for method in data:
        for exc in method["exchanges"]:
            if "Occupation, annual crop" == exc["name"] and exc["amount"] != 0.9:
                return data  # if cfs were already changed

    for method in data:
        for exc in method["exchanges"]:
            exc["amount"] = 100 * (exc["amount"] / total_land_earth)

    return data

In [45]:
si.apply_strategy(change_unit_of_biofootprint_land_use)
si.statistics()

Applying strategy: change_unit_of_biofootprint_land_use
1 methods
37 cfs
0 unlinked cfs


(1, 37, 0)

In [46]:
# check if there are duplicates (both numbers should be equal)
remove_duplicates_if_any(si)

(not PBs) Biological footprint, LU** :  37 37
No duplicates.



In [47]:
si.statistics()

1 methods
37 cfs
0 unlinked cfs


(1, 37, 0)

In [48]:
si.write_excel("BII_land_use-component")

Wrote matching file to:
C:\Users\ViteksPC\AppData\Local\pylca\Brightway3\ecoinvent-36.d2073cbbe9aff3f7cc76c4f9a36d6789\output\lcia-matching-BII_land_use-component.xlsx


In [49]:
PATH_TO_METHODS = os.path.join(
    r"C:\Users\ViteksPC\Documents\00-ETH_projects\AESAmethods\bw_methods"
    f"\\aesa_ChangeBiosphereIntegrity_FunctionalDiversity_Hierarchist.xlsx"
)
print(PATH_TO_METHODS)

C:\Users\ViteksPC\Documents\00-ETH_projects\AESAmethods\bw_methods\aesa_ChangeBiosphereIntegrity_FunctionalDiversity_Hierarchist.xlsx


In [24]:
# Go to excel and apply manual corrections
# store corrected excel to PATH_TO_METHODS

In [56]:
# [method for method in bd.methods if "ReCiPe" in str(method)]

In [53]:
method_category

'ClimateChange_EnergyImbalance'

In [59]:
new = bi.ExcelLCIAImporter(
    PATH_TO_METHODS,
    # name=("AESA (PBs-LCIA)", "climate change", "energy imbalance at top-of-atmosphere"),
    name=("AESA (PBs-LCIA)", "climate change", "atmospheric CO2 concentration"),
    description="here will go the description",
    # unit="Wm-2",
    unit="ppm",
    authors="Ryberg, M. W., Owsianiak, M., Richardson, K., Hauschild, M. Z.",
    doi="10.1016/j.ecolind.2017.12.065",
)

In [60]:
new.apply_strategies()

Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.16 seconds


In [62]:
new.apply_strategy(drop_empty_lines)

Applying strategy: drop_empty_lines


In [63]:
# new.data

In [64]:
new.statistics()

1 methods
42 cfs
0 unlinked cfs


(1, 42, 0)

In [65]:
assert len(list(new.unlinked)) == 0

In [296]:
new.write_methods(overwrite=True, verbose=True)

Wrote 1 LCIA methods with 42 characterization factors


In [275]:
[met for met in bd.methods if "concentr" in str(met).lower()]

[('PBs-LCIA (baseline)', 'Climate change - CO2 concentration'),
 ('AESA', 'trial-CC_CO2concentration')]

In [277]:
l_new = bd.Method(("AESA", "trial-CC_CO2concentration")).load()

In [278]:
l_or = bd.Method(("PBs-LCIA (baseline)", "Climate change - CO2 concentration")).load()

In [279]:
len(l_new)

42

In [280]:
len(l_or)

42

In [285]:
[i for i in l_or if i not in l_new]

[]